<a href="https://colab.research.google.com/github/aimedvedeva/Competition2-nlp-/blob/main/comp2_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd

prompts_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Comp2/prompts_train.csv")
summaries_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Comp2/summaries_train.csv")

In [15]:
len(prompts_df)

4

In [16]:
len(summaries_df)

7165

In [17]:
combined_train_df = pd.merge(summaries_df, prompts_df, how = 'left', on = 'prompt_id')

In [3]:
assert len(combined_train_df) == len(summaries_df)

In [4]:
combined_train_df.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text'],
      dtype='object')

In [5]:
combined_train_df.describe()

,content,wording
count,7165.000000,7165.000000
mean,-0.014853,-0.063072
std,1.043569,1.036048
min,-1.729859,-1.962614
25%,-0.799545,-0.872720
50%,-0.093814,-0.081769
75%,0.499660,0.503833
max,3.900326,4.310693


In [6]:
combined_train_df[(combined_train_df['prompt_id'] == 'ebad26') & (combined_train_df['content'] < -1.4)].iloc[0]['text']

'They would "rub it up with soda to take away the smell, pickle it with a "stronger pickle" to remove the smell, and they would extract the bone and cook the inside to sell them as "boneless hams".'

In [7]:
combined_train_df[(combined_train_df['prompt_id'] == 'ebad26') & (combined_train_df['content'] > 3.4)].iloc[0]['text']

'There were various things that the factory would do to cover up and essentially use the spoiled meat that they had. One manner in which they practiced this was by rubbing the meat with soda to take away the smell, as stated in paragraph two.  The workers would also pickle their meat to get rid of the odor but if it was very pungent, they had a second and much stronger pickle which helped destroy the smell, which the workers called, "giving them thirty per cent." Along with this, they would label the packages in many ways to make it look like it was "original" even if they were made out of meat parts that were not supposed to be used. Another form in which they would use and cover-up spoiled meat would be by getting spoiled whole ham and old sausage with mold from Europe and dosing it with borax and glycerin, where it would then be dumped into hoppers and made over again for packaging. Mice, feces, poisoned bread, dirt, diseases, and sawdust were just a few things that would be process

# Calculate metrics

In [19]:
summaries = combined_train_df['text'].tolist()
prompt_texts = combined_train_df['prompt_text'].tolist()
prompt_questions = combined_train_df['prompt_question'].tolist()

In [18]:
combined_train_df.columns

Index(['student_id', 'prompt_id', 'text', 'content', 'wording',
       'prompt_question', 'prompt_title', 'prompt_text'],
      dtype='object')

#### [BERTScore](https://huggingface.co/spaces/evaluate-metric/bertscore)


> BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity. It has been shown to correlate with human judgment on sentence-level and system-level evaluation. Moreover, BERTScore computes precision, recall, and F1 measure, which can be useful for evaluating different language generation tasks.



##### pips

In [ ]:
pip install evaluate

In [ ]:
pip install bert_score

##### calculation

In [7]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
bert_scores = []
for i in range(len(summaries)):
  print(i)
  score = bertscore.compute(predictions=[summaries[i]], references=[prompt_texts[i]], lang="en")
  print(score)
  bert_scores.append(score)

In [29]:
len(bert_scores)

428

In [ ]:
bert

In [31]:
precisions = [bert_score['precision'] for bert_score in bert_scores]
recalls = [bert_score['recall'] for bert_score in bert_scores]
f1s = [bert_score['f1'] for bert_score in bert_scores]

#### [CharacterScore](https://huggingface.co/spaces/evaluate-metric/character)
> CharacTer is a character-level metric inspired by the translation edit rate (TER) metric. It is defined as the minimum number of character edits required to adjust a hypothesis, until it completely matches the reference, normalized by the length of the hypothesis sentence. CharacTer calculates the character level edit distance while performing the shift edit on word level. Unlike the strict matching criterion in TER, a hypothesis word is considered to match a reference word and could be shifted, if the edit distance between them is below a threshold value. The Levenshtein distance between the reference and the shifted hypothesis sequence is computed on the character level. In addition, the lengths of hypothesis sequences instead of reference sequences are used for normalizing the edit distance, which effectively counters the issue that shorter translations normally achieve lower TER.

##### pips

In [34]:
pip install cer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.9 MB/s eta 0:00:00


##### calculting

In [35]:
import evaluate
character = evaluate.load("character")

In [49]:
character_scores = []
for i in range(len(summaries[:429])):
  character_scores.append(character.compute(references=[prompt_texts[i]], predictions=[summaries[i]]))

In [51]:
cer_scores = [character_score['cer_score'] for character_score in character_scores]

In [52]:
cer_scores

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0

#### [NIST_MT](https://huggingface.co/spaces/evaluate-metric/nist_mt)
> DARPA commissioned NIST to develop an MT evaluation facility based on the BLEU score. The official script used by NIST to compute BLEU and NIST score is mteval-14.pl. The main differences are:
* BLEU uses geometric mean of the ngram overlaps, NIST uses arithmetic mean.
* NIST has a different brevity penalty
* NIST score from mteval-14.pl has a self-contained tokenizer (in the Hugging Face implementation we rely on NLTK’s implementation of the NIST-specific tokenizer)

In [53]:
import evaluate
nist_mt = evaluate.load("nist_mt")

In [54]:
nist_scores = []
for i in range(len(summaries[:429])):
  nist_scores.append(nist_mt.compute(preds=[summaries[i]], refs=[prompt_texts[i]]))

nist_mt_scores = [nist_score['nist_mt'] for nist_score in nist_scores]

ValueError: ignored

#### [Charcut](https://huggingface.co/spaces/evaluate-metric/charcut_mt)
> CharCut compares outputs of MT systems with reference translations. The matching algorithm is based on an iterative search for longest common substrings, combined with a length-based threshold that limits short and noisy character matches. As a similarity metric this is not new, but to the best of our knowledge it was never applied to highlighting and scoring of MT outputs. It has the neat effect of keeping character-based differences readable by humans

##### pips

In [56]:
pip install charcut

##### calculating

In [57]:
import evaluate
charcut = evaluate.load("charcut")

FileNotFoundError: ignored

In [ ]:
results = []
for i in range(len(summaries[:429])):
  results.append(charcut.compute(references=[prompt_texts[i]], predictions=[summaries[i]]))

charcut_scores = [result['charcut_mt'] for result in results]

#### [ExactMatch](https://huggingface.co/spaces/evaluate-metric/exact_match)
> A given predicted string’s exact match score is 1 if it is the exact same as its reference string, and is 0 otherwise.
* Example 1: The exact match score of prediction “Happy Birthday!” is 0, given its reference is “Happy New Year!”.
* Example 2: The exact match score of prediction “The Colour of Magic (1983)” is 1, given its reference is also “The Colour of Magic (1983)”.
The exact match score of a set of predictions is the sum of all of the individual exact match scores in the set, divided by the total number of predictions in the set.
* Example: The exact match score of the set {Example 1, Example 2} (above) is 0.5.



In [58]:
from evaluate import load
exact_match_metric = load("exact_match")

In [97]:
results = []

for i in range(len(summaries[:429])):
  results.append(exact_match_metric.compute(predictions=[summaries[i]], references=[prompt_texts[i]]))

exact_match_scores = [result['exact_match'] for result in results]

In [98]:
exact_match_scores

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

#### [RougeScore](https://huggingface.co/spaces/evaluate-metric/rouge)

> ROUGE, or Recall-Oriented Understudy for Gisting Evaluation, is a set of metrics and a software package used for evaluating automatic summarization and machine translation software in natural language processing. The metrics compare an automatically produced summary or translation against a reference or a set of references (human-produced) summary or translation.
Note that ROUGE is case insensitive, meaning that upper case letters are treated the same way as lower case letters.

##### pips

In [62]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=bc4f350ead43162d49b4b2874d74402fa271d916beb67a4589d6e38a66402f86
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


##### calculating

In [63]:
rouge = evaluate.load('rouge')

In [105]:
results = []

for i in range(len(summaries[:428])):
  results.append(rouge.compute(predictions=[summaries[i]], references=[prompt_texts[i]], use_aggregator=False))

rouge1_scores = [result['rouge1'] for result in results]
rouge2_scores = [result['rouge2'] for result in results]
rougeL_scores = [result['rougeL'] for result in results]
rougeLsum_scores = [result['rougeLsum'] for result in results]

In [69]:
rougeLsum_scores

[[0.0755287009063444],
 [0.08163265306122448],
 [0.32955974842767294],
 [0.0684931506849315],
 [0.2109181141439206],
 [0.04901960784313726],
 [0.10062893081761008],
 [0.056695992179863146],
 [0.10670731707317073],
 [0.035143769968051124],
 [0.0273972602739726],
 [0.11797235023041476],
 [0.1726844583987441],
 [0.10381679389312977],
 [0.09653092006033183],
 [0.06305418719211822],
 [0.09900990099009901],
 [0.17543859649122806],
 [0.1447562776957164],
 [0.10030395136778114],
 [0.04140127388535032],
 [0.09090909090909091],
 [0.05732484076433121],
 [0.40813008130081296],
 [0.20451127819548873],
 [0.14081826831588964],
 [0.3718887262079063],
 [0.04786324786324786],
 [0.14550641940085593],
 [0.1664050235478807],
 [0.1008130081300813],
 [0.22988505747126436],
 [0.07278481012658228],
 [0.09137055837563453],
 [0.13979496738117428],
 [0.03180914512922465],
 [0.0641891891891892],
 [0.20089955022488754],
 [0.1325648414985591],
 [0.20357634112792297],
 [0.11526479750778816],
 [0.06191950464396285],
 

#### [CometScore](https://huggingface.co/spaces/evaluate-metric/wer)


> Crosslingual Optimized Metric for Evaluation of Translation (COMET) is an open-source framework used to train Machine Translation metrics that achieve high levels of correlation with different types of human judgments.

*COMET takes 3 lists of strings as input: sources (a list of source sentences), predictions (a list of candidate translations) and references (a list of reference translations).*

##### pips

In [ ]:
pip install unbabel-comet

##### calculating

In [72]:
from evaluate import load
comet_metric = load('comet')

wmt20-comet-da.tar.gz: 1.79GB [00:47, 37.4MB/s]                            
INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v1.9.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../root/.cache/torch/unbabel_comet/wmt20-comet-da/checkpoints/model.ckpt`


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:255: UserWarning: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
  rank_zero_warn(


In [ ]:
results = []
for i in range(len(summaries[:429])):
  result = comet_metric.compute(predictions=[summaries[i]], references=[prompt_questions[i]], sources=[prompt_texts[i]])
  comet_score = [v for v in result["scores"]]
  results.append(comet_score)

In [78]:
comet_metric.compute(predictions=[summaries[i]], references=[prompt_questions[i]], sources=[prompt_texts[i]])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


{'mean_score': 'system_score', 'scores': 'scores'}

#### [MAUVE](https://huggingface.co/spaces/evaluate-metric/mauve)
> MAUVE is a measure of the gap between neural text and human text. It is computed using the Kullback–Leibler (KL) divergences between the two distributions of text in a quantized embedding space of a large language model. MAUVE can identify differences in quality arising from model sizes and decoding algorithms.



##### pips

In [82]:
pip install mauve-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 25.0 MB/s eta 0:00:00


##### calculating

In [83]:
from evaluate import load
mauve = load('mauve')

In [85]:
mauve_results = []

for i in range(len(summaries[:429])):
  mauve_results.append(mauve.compute(predictions=[summaries[i]], references=[prompt_texts[i]]))

Loading tokenizer
Tokenizing text...
Loading tokenizer
Loading model


Featurizing tokens


Featurizing p:   0%|          | 0/1 [00:00<?, ?it/s]

Tokenizing text...
Featurizing tokens


Featurizing q:   0%|          | 0/1 [00:00<?, ?it/s]

...

seed = 25
performing clustering in lower dimension = 0
kmeans time: 0.0 s
total discretization time: 0.0 seconds
Tokenizing text...
Featurizing tokens


Featurizing p:   0%|          | 0/1 [00:00<?, ?it/s]

Tokenizing text...
Featurizing tokens


Featurizing q:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
mauve_scores = [item['mauve'] for item in mauve_results]

#### [MeteorScore](https://huggingface.co/spaces/evaluate-metric/meteor)

> METEOR (Metric for Evaluation of Translation with Explicit ORdering) is a machine translation evaluation metric, which is calculated based on the harmonic mean of precision and recall, with recall weighted more than precision.
METEOR is based on a generalized concept of unigram matching between the machine-produced translation and human-produced reference translations. Unigrams can be matched based on their surface forms, stemmed forms, and meanings. Once all generalized unigram matches between the two strings have been found, METEOR computes a score for this matching using a combination of unigram-precision, unigram-recall, and a measure of fragmentation that is designed to directly capture how well-ordered the matched words in the machine translation are in relation to the reference.



##### pips

In [87]:
pip install meteor

In [88]:
meteor = evaluate.load('meteor')

ImportError: ignored

In [ ]:
meteor_scores = []

for i in range(len(summaries)):
  meteor_scores.append(meteor.compute(predictions=[summaries[i]], references=[prompt_texts[i]]))

In [ ]:
meteor_scores = [item['meteor'] for item in meteor_scores]

#### [SariScore](https://huggingface.co/spaces/evaluate-metric/sari)


> SARI (system output against references and against the input sentence) is a metric used for evaluating automatic text simplification systems.
The metric compares the predicted simplified sentences against the reference and the source sentences. It explicitly measures the goodness of words that are added, deleted and kept by the system.

SARI can be computed as:

sari = ( F1_add + F1_keep + P_del) / 3

where

F1_add is the n-gram F1 score for add operations

F1_keep is the n-gram F1 score for keep operations

P_del is the n-gram precision score for delete operations

The number of n grams, n, is equal to 4, as in the original paper.



In [90]:
pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=e4013c110340131b609a657810f94f1374bc6b2eb8828b2d1854a28747dff5bf
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [91]:
from evaluate import load
sari = load("sari")

The metric takes 3 inputs: sources (a list of source sentence strings), predictions (a list of predicted sentence strings) and references (a list of lists of reference sentence strings)

In [93]:
sari_scores = []
for i in range(len(summaries[:429])):
  sari_scores.append(sari.compute(sources=[prompt_texts[i]], predictions=[summaries[i]], references=[prompt_questions[i]]))

sari_scores = [item['sari'] for item in sari_scores]

ValueError: ignored

#### SQUAD/CUAD TODO

[CuadScore](https://huggingface.co/spaces/evaluate-metric/cuad)


> This metric wraps the official scoring script for version 1 of the Contract Understanding Atticus Dataset (CUAD), which is a corpus of more than 13,000 labels in 510 commercial legal contracts that have been manually labeled to identify 41 categories of important clauses that lawyers look for when reviewing contracts in connection with corporate transactions.
The CUAD metric computes several scores: Exact Match, F1 score, Area Under the Precision-Recall Curve, Precision at 80% recall and Precision at 90% recall.



In [ ]:
from evaluate import load
cuad_metric = load("cuad")

**The CUAD metric takes two inputs :**

*predictions*, a list of question-answer dictionaries with the following key-values:

* id: the id of the question-answer pair as given in the references.
* prediction_text: a list of possible texts for the answer, as a list of strings depending on a threshold on the confidence probability of each prediction.

*references*: a list of question-answer dictionaries with the following key-values:

* id: the id of the question-answer pair (the same as above).
answers: a dictionary in the CUAD dataset format with the following keys:
* text: a list of possible texts for the answer, as a list of strings.
answer_start: a list of start positions for the answer, as a list of ints.
Note that answer_start values are not taken into account to compute the metric.

In [ ]:
predictions = [{'prediction_text': ['The seller:', 'The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd.'], 'id': 'LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement__Parties'}]
references = [{'answers': {'answer_start': [143, 49], 'text': ['The seller:', 'The buyer/End-User: Shenzhen LOHAS Supply Chain Management Co., Ltd.']}, 'id': 'LohaCompanyltd_20191209_F-1_EX-10.16_11917878_EX-10.16_Supply Agreement__Parties'}]
results = cuad_metric.compute(predictions=predictions, references=references)
print(results)
{'exact_match': 100.0, 'f1': 100.0, 'aupr': 0.0, 'prec_at_80_recall': 1.0, 'prec_at_90_recall': 1.0}

#### Build the final dataset

In [112]:
dataset = pd.DataFrame(columns = ['precision', 'recall', 'f1', #'cer_score', 'nist_mt_score', 'charcut_mt_score', 'meteor', 'sari',
                                  #'exact_match_score','mauve_score',
                                  'rouge1_score', 'rouge2_score', 'rougeL_score', 'rougeLsum_score', #'comet_score'\
                                   'content', 'wording'], data=[])
# all columns except wording and content are features
# content and wording are target values
# we gonna train a regression model to predict content and wording

In [113]:
dataset['precision'] = precisions
dataset['recall'] = recalls
dataset['f1'] = f1s
#dataset['cer_score'] = cer_scores  # couse it's a constant
#dataset['nist_mt_score'] = nist_mt_scores   # error
#dataset['charcut_mt_score'] = charcut_scores  # error
#dataset['exact_math_score'] = exact_match_scores  # couse it's a constant
dataset['rouge1_score'] = rouge1_scores
dataset['rouge2_score'] = rouge2_scores
dataset['rougeL_score'] = rougeL_scores
dataset['rougeLsum_score'] = rougeLsum_scores
#dataset['comet_score'] = comet_scores  # error
#dataset['mauve_score'] = mauve_scores  #too long to wait
#dataset['meteor'] = meteor_scores  # error
#dataset['sari'] = sari_scores   #error
dataset['content'] = combined_train_df['content']
dataset['wording'] = combined_train_df['wording']

#### Observe dataset

In [114]:
dataset

,precision,recall,f1,rouge1_score,rouge2_score,rougeL_score,rougeLsum_score,content,wording
0,[0.8283703923225403],[0.7932977080345154],[0.8104547262191772],[0.1027190332326284],[0.021212121212121213],[0.06948640483383685],[0.0755287009063444],0.205683,0.380538
1,[0.8546106219291687],[0.7895169258117676],[0.8207751512527466],[0.09523809523809523],[0.044790652385589096],[0.0738581146744412],[0.08163265306122448],-0.548304,0.506755
2,[0.8588669896125793],[0.8438898324966431],[0.851312518119812],[0.38993710691823896],[0.10340479192938208],[0.19874213836477986],[0.32955974842767294],3.128928,4.231226
3,[0.8455113768577576],[0.7886843085289001],[0.816109836101532],[0.0684931506849315],[0.020618556701030927],[0.0547945205479452],[0.0684931506849315],-0.210614,-0.471415
4,[0.8321478366851807],[0.8212687969207764],[0.8266725540161133],[0.2878411910669975],[0.0945273631840796],[0.1563275434243176],[0.2109181141439206],3.272894,3.219757
...,...,...,...,...,...,...,...,...,...
423,[0.8455602526664734],[0.7928799986839294],[0.8183732032775879],[0.06470588235294117],[0.021611001964636542],[0.05098039215686275],[0.04901960784313726],-0.921625,-1.213611
424,[0.8645575642585754],[0.8127844929695129],[0.8378720283508301],[0.17279411764705882],[0.09208103130755066],[0.1286764705882353],[0.14338235294117646],1.002762,0.669076
425,[0.8405666947364807],[0.7830688953399658],[0.810799777507782],[0.09597523219814241],[0.031055900621118012],[0.07430340557275542],[0.086687306501548],-0.456956,-0.042516
426,[0.8815320730209351],[0.7830352783203125],[0.8293695449829102],[0.053465346534653464],[0.03968253968253968],[0.049504950495049514],[0.053465346534653464],-1.109220,-1.384700


In [119]:
for column in dataset.drop(columns = ['content', 'wording']).columns:
  dataset[column] = dataset[column].apply(lambda x: x[0])

In [120]:
dataset

,precision,recall,f1,rouge1_score,rouge2_score,rougeL_score,rougeLsum_score,content,wording
0,0.828370,0.793298,0.810455,0.102719,0.021212,0.069486,0.075529,0.205683,0.380538
1,0.854611,0.789517,0.820775,0.095238,0.044791,0.073858,0.081633,-0.548304,0.506755
2,0.858867,0.843890,0.851313,0.389937,0.103405,0.198742,0.329560,3.128928,4.231226
3,0.845511,0.788684,0.816110,0.068493,0.020619,0.054795,0.068493,-0.210614,-0.471415
4,0.832148,0.821269,0.826673,0.287841,0.094527,0.156328,0.210918,3.272894,3.219757
...,...,...,...,...,...,...,...,...,...
423,0.845560,0.792880,0.818373,0.064706,0.021611,0.050980,0.049020,-0.921625,-1.213611
424,0.864558,0.812784,0.837872,0.172794,0.092081,0.128676,0.143382,1.002762,0.669076
425,0.840567,0.783069,0.810800,0.095975,0.031056,0.074303,0.086687,-0.456956,-0.042516
426,0.881532,0.783035,0.829370,0.053465,0.039683,0.049505,0.053465,-1.109220,-1.384700


In [121]:
dataset.to_csv('dataset_first_428.csv')